<h3>Scrap the books (name, price, rate) for each category and put them into a CSV file
</h3> <br> <a href="https://books.toscrape.com/">Books Site</a> 
<br>


In [1]:
import requests
import csv
from bs4 import BeautifulSoup

In [2]:
url='https://books.toscrape.com/index.html'
response = requests.request('GET', url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
page_books = soup.find_all('article', attrs={'class': 'product_pod'})
len(page_books)

20

In [4]:
book_name = page_books[0].find('h3').find('a').get('title')
print(book_name)

price = float(page_books[0].find('p', attrs={'class': 'price_color'}).get_text().split('£')[1])
print(price)

rating = page_books[12].find('p').get('class')[1]
print(rating)

A Light in the Attic
51.77
Five


In [5]:
def get_all_category(home_link):
    response = requests.request('GET', home_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    categories = {}
    all_books_category_links = soup.find('div', attrs={'class': 'side_categories'}).find('ul').find('li').find('ul').find_all('a')
    #all_books_category_links
    for link in all_books_category_links:
        value = home_link+link.get('href')
        category_name = link.get_text().strip()
        categories[category_name] = value
    return categories
    

In [6]:
get_all_category('http://books.toscrape.com/')

{'Travel': 'http://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'Mystery': 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'Historical Fiction': 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'Sequential Art': 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'Classics': 'http://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'Philosophy': 'http://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'Romance': 'http://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'Womens Fiction': 'http://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'Fiction': 'http://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'Childrens': 'http://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'Religion': 'http://books.toscrape.com/catalogue/category/books/rel

In [7]:
def get_books_Count_by_category(page_link):
    response = requests.request('GET', page_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    books_count = soup.find('form', attrs={'class': 'form-horizontal'}).find_all('strong')[0].get_text()
    return books_count

In [8]:
def get_books_by_page(page_link,writer):
    count=1
    pagination =True
    while pagination:
        response = requests.request('GET', page_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        books = soup.find_all('article', attrs={'class': 'product_pod'})
        for book in books:
            name = book.find('h3').find('a').get('title').strip()
            rating = (book.find('p').get('class')[1]).strip()
            price = float(book.find('p', attrs={'class': 'price_color'}).get_text().split('£')[1])
            writer.writerow({'Book_Name': name, 'Price': price,'Rating':rating})
            print(f'no :{count}  Book name: {name}')
            count=count+1
        next_url=soup.find('li',attrs={'class':'next'})
        if next_url == None:
            pagination=False
        else:
            next_url=soup.find('li',attrs={'class':'next'}).find('a').get('href')
            url='https://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html'
            new_url = page_link[:page_link.rfind("/")]
            page_link= new_url+'/'+next_url

In [9]:
categories = get_all_category('http://books.toscrape.com/')
with open('Books.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['Book_Name','Price', 'Rating'])
    writer.writeheader()
    for category, url in categories.items(): 
        print('')
        print(f'Category : {category}  no of books : {get_books_Count_by_category(url)}')
        print(url)
        print('')
        get_books_by_page(url,writer)


Category : Travel  no of books : 11
http://books.toscrape.com/catalogue/category/books/travel_2/index.html

no :1  Book name: It's Only the Himalayas
no :2  Book name: Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond
no :3  Book name: See America: A Celebration of Our National Parks & Treasured Sites
no :4  Book name: Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel
no :5  Book name: Under the Tuscan Sun
no :6  Book name: A Summer In Europe
no :7  Book name: The Great Railway Bazaar
no :8  Book name: A Year in Provence (Provence #1)
no :9  Book name: The Road to Little Dribbling: Adventures of an American in Britain (Notes From a Small Island #2)
no :10  Book name: Neither Here nor There: Travels in Europe
no :11  Book name: 1,000 Places to See Before You Die

Category : Mystery  no of books : 32
http://books.toscrape.com/catalogue/category/books/mystery_3/index.html

no :1  Book name: Sharp Objects
no :2  Book name: In a Dark, Dark Wood
no :3  Book 

Category : Classics  no of books : 19
http://books.toscrape.com/catalogue/category/books/classics_6/index.html

no :1  Book name: The Secret Garden
no :2  Book name: The Metamorphosis
no :3  Book name: The Pilgrim's Progress
no :4  Book name: The Hound of the Baskervilles (Sherlock Holmes #5)
no :5  Book name: Little Women (Little Women #1)
no :6  Book name: Gone with the Wind
no :7  Book name: Candide
no :8  Book name: Animal Farm
no :9  Book name: Wuthering Heights
no :10  Book name: The Picture of Dorian Gray
no :11  Book name: The Complete Stories and Poems (The Works of Edgar Allan Poe [Cameo Edition])
no :12  Book name: Beowulf
no :13  Book name: And Then There Were None
no :14  Book name: The Story of Hong Gildong
no :15  Book name: The Little Prince
no :16  Book name: Sense and Sensibility
no :17  Book name: Of Mice and Men
no :18  Book name: Emma
no :19  Book name: Alice in Wonderland (Alice's Adventures in Wonderland #1)

Category : Philosophy  no of books : 11
http://books.t

no :21  Book name: The Cat in the Hat (Beginner Books B-1)
no :22  Book name: Red: The True Story of Red Riding Hood
no :23  Book name: Horrible Bear!
no :24  Book name: Green Eggs and Ham (Beginner Books B-16)
no :25  Book name: Counting Thyme
no :26  Book name: Are We There Yet?
no :27  Book name: Diary of a Minecraft Zombie Book 1: A Scare of a Dare (An Unofficial Minecraft Book)
no :28  Book name: Matilda
no :29  Book name: Charlie and the Chocolate Factory (Charlie Bucket #1)

Category : Religion  no of books : 7
http://books.toscrape.com/catalogue/category/books/religion_12/index.html

no :1  Book name: Don't Be a Jerk: And Other Practical Advice from Dogen, Japan's Greatest Zen Master
no :2  Book name: You Are What You Love: The Spiritual Power of Habit
no :3  Book name: God: The Most Unpleasant Character in All Fiction
no :4  Book name: The Book of Mormon
no :5  Book name: A History of God: The 4,000-Year Quest of Judaism, Christianity, and Islam
no :6  Book name: The Bhagavad 

no :101  Book name: Black Flags: The Rise of ISIS
no :102  Book name: Are We Smart Enough to Know How Smart Animals Are?
no :103  Book name: A Walk in the Woods: Rediscovering America on the Appalachian Trail
no :104  Book name: The Suffragettes (Little Black Classics, #96)
no :105  Book name: Kindle Paperwhite User's Guide
no :106  Book name: H is for Hawk
no :107  Book name: Travels with Charley: In Search of America
no :108  Book name: The Tumor
no :109  Book name: The End of the Jesus Era (An Investigation #1)
no :110  Book name: Eat, Pray, Love

Category : Music  no of books : 13
http://books.toscrape.com/catalogue/category/books/music_14/index.html

no :1  Book name: Rip it Up and Start Again
no :2  Book name: Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991
no :3  Book name: How Music Works
no :4  Book name: Love Is a Mix Tape (Music #1)
no :5  Book name: Please Kill Me: The Uncensored Oral History of Punk
no :6  Book name: Kill 'Em and Leave: S

no :121  Book name: The Giver (The Giver Quartet #1)
no :122  Book name: The Girl Who Played with Fire (Millennium Trilogy #2)
no :123  Book name: The Demon-Haunted World: Science as a Candle in the Dark
no :124  Book name: The Book Thief
no :125  Book name: The Autobiography of Malcolm X
no :126  Book name: Shopaholic & Baby (Shopaholic #5)
no :127  Book name: Quiet: The Power of Introverts in a World That Can't Stop Talking
no :128  Book name: Packing for Mars: The Curious Science of Life in the Void
no :129  Book name: Orange Is the New Black
no :130  Book name: One for the Money (Stephanie Plum #1)
no :131  Book name: Morning Star (Red Rising #3)
no :132  Book name: Life After Life
no :133  Book name: Lean In: Women, Work, and the Will to Lead
no :134  Book name: Is Everyone Hanging Out Without Me? (And Other Concerns)
no :135  Book name: Gone Girl
no :136  Book name: Fellside
no :137  Book name: Eclipse (Twilight #3)
no :138  Book name: Dracula
no :139  Book name: Dead Wake: The L

no :21  Book name: Every Heart a Doorway (Every Heart A Doorway #1)
no :22  Book name: A Gathering of Shadows (Shades of Magic #2)
no :23  Book name: The Raven Boys (The Raven Cycle #1)
no :24  Book name: The False Prince (The Ascendance Trilogy #1)
no :25  Book name: Tell the Wind and Fire
no :26  Book name: A Feast for Crows (A Song of Ice and Fire #4)
no :27  Book name: The Demonists (Demonist #1)
no :28  Book name: The Beast (Black Dagger Brotherhood #14)
no :29  Book name: Paper and Fire (The Great Library #2)
no :30  Book name: Harry Potter and the Order of the Phoenix (Harry Potter #5)
no :31  Book name: Harry Potter and the Half-Blood Prince (Harry Potter #6)
no :32  Book name: Harry Potter and the Chamber of Secrets (Harry Potter #2)
no :33  Book name: The Rose & the Dagger (The Wrath and the Dawn #2)
no :34  Book name: Soldier (Talon #3)
no :35  Book name: Midnight Riot (Peter Grant/ Rivers of London - books #1)
no :36  Book name: Heir to the Sky
no :37  Book name: Eragon (Th

no :1  Book name: The Argonauts
no :2  Book name: M Train
no :3  Book name: Lab Girl
no :4  Book name: Approval Junkie: Adventures in Caring Too Much
no :5  Book name: Running with Scissors
no :6  Book name: Me Talk Pretty One Day
no :7  Book name: Lust & Wonder
no :8  Book name: Life Without a Recipe
no :9  Book name: A Heartbreaking Work of Staggering Genius

Category : Parenting  no of books : 1
http://books.toscrape.com/catalogue/category/books/parenting_28/index.html

no :1  Book name: Catastrophic Happiness: Finding Joy in Childhood's Messy Years

Category : Adult Fiction  no of books : 1
http://books.toscrape.com/catalogue/category/books/adult-fiction_29/index.html

no :1  Book name: Fifty Shades Freed (Fifty Shades #3)

Category : Humor  no of books : 10
http://books.toscrape.com/catalogue/category/books/humor_30/index.html

no :1  Book name: The Long Haul (Diary of a Wimpy Kid #9)
no :2  Book name: Old School (Diary of a Wimpy Kid #10)
no :3  Book name: I Know What I'm Doing -

no :1  Book name: Louisa: The Extraordinary Life of Mrs. Adams
no :2  Book name: Setting the World on Fire: The Brief, Astonishing Life of St. Catherine of Siena
no :3  Book name: The Faith of Christopher Hitchens: The Restless Soul of the World's Most Notorious Atheist
no :4  Book name: Benjamin Franklin: An American Life
no :5  Book name: The Rise of Theodore Roosevelt (Theodore Roosevelt #1)

Category : Thriller  no of books : 11
http://books.toscrape.com/catalogue/category/books/thriller_37/index.html

no :1  Book name: In Her Wake
no :2  Book name: The Elephant Tree
no :3  Book name: Behind Closed Doors
no :4  Book name: You (You #1)
no :5  Book name: The Guilty (Will Robie #4)
no :6  Book name: The 14th Colony (Cotton Malone #11)
no :7  Book name: Give It Back
no :8  Book name: Killing Floor (Jack Reacher #1)
no :9  Book name: The Bone Hunters (Lexy Vaughan & Steven Macaulay #2)
no :10  Book name: Far From True (Promise Falls Trilogy #2)
no :11  Book name: The Travelers

Category

## End Work!
### By Abdulla Elshourbagy.